## Exploratory Visualizations
- Look at distributions.
- Look at correlations.
- Look at relationships to target (scatter plots for continuous, box plots for categorical).

## Pre-processing
- One-hot encode categorical variables.
- Train/test split your data.
- Scale your data.
- Consider using automated feature selection.

In [ ]:
# Creating our features list; note that ID is not among them.
# Pandas and Numpy will preserve the order of predictions on its own.
features = ['Lot Area', 'Yr Sold', 'Overall Qual', 'Year Remod/Add', 'Year Built']

In [ ]:
# Get correlation of variables.
corr = df.corr()
print(corr)
# Set up mask to be "True" in the upper triangle.
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
# Plot our correlation heatmap, while masking the upper triangle to be white.
sns.heatmap(corr, mask = mask, cmap = 'mako', annot = True, vmin=-1, vmax=1);

In [ ]:
# Setting up our features and our target from the train_df to feed into a linear regression.
X = train_df[features]
y_actualdf = train_df[['SalePrice']]
y_actual   = train_df['SalePrice']

In [ ]:
# Check that we have an equal number of observations in our X and y.
X.shape[0] == y_actualdf.shape[0]
# Verify dimensions, n = No. of rows
print('X:        ', X.shape) # X.shape equals (n,p)
print('y_actual: ', y_actual.shape) # y.shape equals (n, null)

In [ ]:
# Check that everything is copacetic.
X.head()